In [13]:
import pandas as pd
import numpy as np
data = pd.read_csv('data/data.csv')
#data = pd.DataFrame.from_csv('data/data.csv')

## Get Id's where Age is 0 

In [14]:
zeroAge = data[data['Age'] == 0].index.tolist()

In [15]:
runnerId = [data.iloc[x]['Id'] for x in zeroAge]

In [16]:
for n in runnerId:
    print data[data['Id'] == n]

      Id  Age  Rank  Year   Time  Pace  Sex
488  308   45  1204  2016  15137   578    0
489  308    0  1357  2014  15975   609    0
        Id  Age  Rank  Year   Time  Pace  Sex
2041  1229   48  2717  2009  22803   870    1
2042  1229   47  2414  2008  20344   776    1
2043  1229   53  3194  2014  21215   809    1
2044  1229   46  2054  2007  19896   759    1
2045  1229   51  3870  2012  23302   889    1
2046  1229    0  3332  2011  20990   801    1
2047  1229   50  2208  2006  21090   804    1
2048  1229   50  3482  2011  22896   873    1
2049  1229   49  2811  2010  23191   885    1
2050  1229   44  1566  2005  19757   754    1
2051  1229   55  2783  2016  20941   800    1
2052  1229   50  2748  2015  25435   970    1
         Id  Age  Rank  Year   Time  Pace  Sex
10743  6491   43   217  2006  12496   476    1
10744  6491   50   396  2015  13172   502    1
10745  6491    0   264  2012  12452   475    1
10746  6491   42    84  2005  11596   442    1
         Id  Age  Rank  Year   Time

The following runner id's represent instances where the age is unknown, but the id only appears once, therefore age cannot be corrected: [16673,16954,17902,18202,22942]. To deal with these, the unknown age of the id was set to the average age of all runners who ranked within a range of +/- 50 of the runner in that particular year.

In [17]:
def correctSingleInstanceAge(runnerId,data):
    rank = int(data[data['Id'] == runnerId]['Rank'])
    year = int(data[data['Id'] == runnerId]['Year'])
    rangeOfRunners = data[(data['Rank']>rank-50) & (data['Rank']<rank+50)& (data['Year'] == year)]
    avgAge = rangeOfRunners['Age'].sum()/rangeOfRunners['Age'].size

    index = data[data['Id']==runnerId].index
    data.set_value(index, 'Age', avgAge)
    return


In [18]:
ageCorrectionRunnerIds = [16673,16954,17902,18202,22942]
for n in ageCorrectionRunnerIds:
    correctSingleInstanceAge(n,data)

Runner id's [23714,26270] represent unknown identities, despite clearly being different people that ran in the same year. This represents 6 people in 2013 and 4 people in 2014.

In [19]:
zeroAge = data[(data['Age'] == 0)&(data['Id'] != 26270) &(data['Id'] != 23714) ].index.tolist()
runnerId = [data.iloc[x]['Id'] for x in zeroAge]
for i in runnerId:
    print data[data['Id'] == i].sort_values(by='Year')

      Id  Age  Rank  Year   Time  Pace  Sex
489  308    0  1357  2014  15975   609    0
488  308   45  1204  2016  15137   578    0
        Id  Age  Rank  Year   Time  Pace  Sex
2050  1229   44  1566  2005  19757   754    1
2047  1229   50  2208  2006  21090   804    1
2044  1229   46  2054  2007  19896   759    1
2042  1229   47  2414  2008  20344   776    1
2041  1229   48  2717  2009  22803   870    1
2049  1229   49  2811  2010  23191   885    1
2046  1229    0  3332  2011  20990   801    1
2048  1229   50  3482  2011  22896   873    1
2045  1229   51  3870  2012  23302   889    1
2043  1229   53  3194  2014  21215   809    1
2052  1229   50  2748  2015  25435   970    1
2051  1229   55  2783  2016  20941   800    1
         Id  Age  Rank  Year   Time  Pace  Sex
10746  6491   42    84  2005  11596   442    1
10743  6491   43   217  2006  12496   476    1
10745  6491    0   264  2012  12452   475    1
10744  6491   50   396  2015  13172   502    1
         Id  Age  Rank  Year   Time

For the remaining runner Ids, the participant ran in at least one other year where the age information is recorded. By taking the difference in the years, this offset is added to the recorded age.

In [20]:
def offsetAge(idRecorded,idError,data):
    ageRecorded = data.loc[idRecorded]['Age']
    yearRecorded = data.loc[idRecorded]['Year']
    yearError = data.loc[idError]['Year']
    offset = yearError - yearRecorded
    data.set_value(idError, 'Age', ageRecorded+offset)
    return

In [21]:
offsetAge(488,489,data)
offsetAge(2049,2046,data)
offsetAge(10743,10745,data)
offsetAge(13066,13065,data)
offsetAge(24232,24231,data)

In [22]:
data[(data['Id'] != 23714) & (data['Id'] != 26270) & (data['Age']==0)]

,Id,Age,Rank,Year,Time,Pace,Sex


## Smallest Age

In [23]:
data[(data['Id'] != 23714) & (data['Id'] != 26270)]['Age'].min()

9

In [26]:
data[data['Age']==9]

,Id,Age,Rank,Year,Time,Pace,Sex
31786,23689,9,2339,2013,6994,533,1


In [27]:
data[data['Id']==23689]

,Id,Age,Rank,Year,Time,Pace,Sex
31786,23689,9,2339,2013,6994,533,1
31787,23689,45,2339,2013,6994,533,1


Participants ran in the same year with the same id/time/pace/sex. Id 31786 can be perceived as an error and eliminated.

In [33]:
data = data.drop([31786])

In [36]:
data[(data['Id'] != 23714) & (data['Id'] != 26270)]['Age'].min()

10

## Force Ages to increments of 5

In [11]:
for i in range (data['Age'].size):
    data.iloc[i]['Age'] = data.iloc[i]['Age'] - data.iloc[i]['Age']%5

In [12]:
data[(data['Id'] != 23714) & (data['Id'] != 26270)]['Age'].min()

5

In [1]:
import matplotlib.pyplot as plt
pd.Series.hist(data['Age'])
#plt.show()

NameError: name 'pd' is not defined

## Add weather data